# 多人龙与地下城本笔记本展示了`DialogueAgent`和`DialogueSimulator`类如何使得将[双人龙与地下城示例](https://python.langchain.com/en/latest/use_cases/agent_simulations/two_player_dnd.html)扩展到多个玩家变得容易。模拟两个玩家和多个玩家之间的主要区别在于调整每个代理人发言的时间表。为此，我们增加了`DialogueSimulator`类，以接受一个自定义函数来确定每个代理人发言的时间表。在下面的示例中，每个角色按照轮流的方式发言，每个玩家之间插入一个讲故事的人。

## 导入LangChain相关模块

In [1]:
from typing import Callable, List# 导入需要的模块from langchain.schema import (    HumanMessage,  # 导入HumanMessage类    SystemMessage,  # 导入SystemMessage类)from langchain_openai import ChatOpenAI  # 导入ChatOpenAI类

## `DialogueAgent` 类`DialogueAgent` 类是对 `ChatOpenAI` 模型的简单封装，它通过简单地将消息作为字符串连接来存储 `dialogue_agent` 视角下的消息历史。它公开了两种方法：- `send()`: 将聊天模型应用于消息历史并返回消息字符串- `receive(name, message)`: 将由 `name` 说出的 `message` 添加到消息历史中

In [2]:
class DialogueAgent:    def __init__(        self,        name: str,        system_message: SystemMessage,        model: ChatOpenAI,    ) -> None:        self.name = name        self.system_message = system_message        self.model = model        self.prefix = f"{self.name}: "  # 设置对话代理的前缀        self.reset()  # 重置对话历史    def reset(self):        self.message_history = ["Here is the conversation so far."]  # 初始化对话历史列表，包含一条初始消息    def send(self) -> str:        """        将对话模型应用于对话历史，并返回消息字符串        """        message = self.model.invoke(            [                self.system_message,                HumanMessage(content="\n".join(self.message_history + [self.prefix])),            ]        )  # 将系统消息和人类消息作为输入，调用对话模型生成回复消息        return message.content    def receive(self, name: str, message: str) -> None:        """        将{name}说的{message}连接到对话历史中        """        self.message_history.append(f"{name}: {message}")  # 将{name}说的{message}添加到对话历史列表中

## `DialogueSimulator` 类`DialogueSimulator` 类接受一个代理人列表。在每一步中，它执行以下操作：1. 选择下一个发言者2. 调用下一个发言者发送消息3. 将消息广播给所有其他代理人4. 更新步骤计数器。下一个发言者的选择可以实现为任何函数，但在这种情况下，我们只是简单地循环遍历代理人。

In [3]:
class DialogueSimulator:    def __init__(        self,        agents: List[DialogueAgent],        selection_function: Callable[[int, List[DialogueAgent]], int],    ) -> None:        self.agents = agents        self._step = 0        self.select_next_speaker = selection_function    def reset(self):        for agent in self.agents:            agent.reset()    def inject(self, name: str, message: str):        """        用{name}发送的{message}开始对话        """        for agent in self.agents:            agent.receive(name, message)        # 增加时间步        self._step += 1    def step(self) -> tuple[str, str]:        # 1. 选择下一个发言者        speaker_idx = self.select_next_speaker(self._step, self.agents)        speaker = self.agents[speaker_idx]        # 2. 下一个发言者发送消息        message = speaker.send()        # 3. 所有人接收消息        for receiver in self.agents:            receiver.receive(speaker.name, message)        # 4. 增加时间步        self._step += 1        return speaker.name, message

## 定义角色和任务

In [4]:
# 定义角色名称列表character_names = ["Harry Potter", "Ron Weasley", "Hermione Granger", "Argus Filch"]# 定义故事讲述者名称storyteller_name = "Dungeon Master"# 定义任务quest = "Find all of Lord Voldemort's seven horcruxes."# 定义任务头脑风暴的字数限制word_limit = 50  # 任务头脑风暴的字数限制

## 请一个LLM为游戏描述增加细节

In [5]:
game_description = f"""这是一个龙与地下城游戏的主题: {quest}.        角色有: {*character_names,}.        故事由讲故事者 {storyteller_name} 叙述。"""player_descriptor_system_message = SystemMessage(    content="您可以为龙与地下城玩家的描述添加细节。")def generate_character_description(character_name):    character_specifier_prompt = [        player_descriptor_system_message,        HumanMessage(            content=f"""{game_description}            请用不超过 {word_limit} 个字回答以下问题: 请为角色 {character_name} 提供一个创意描述。             直接对 {character_name} 说话。            不要添加其他内容。"""        ),    ]    character_description = ChatOpenAI(temperature=1.0)(        character_specifier_prompt    ).content    return character_descriptiondef generate_character_system_message(character_name, character_description):    return SystemMessage(        content=(            f"""{game_description}    您的名字是 {character_name}.     您的角色描述如下: {character_description}.    您将提出您计划采取的行动，{storyteller_name} 将解释当您采取这些行动时会发生什么。    请以第一人称从 {character_name} 的角度说话。    如果要描述自己的身体动作，请用 '*' 将描述括起来。    不要改变角色！    不要以其他人的视角说话。    记住您是 {character_name}。    在您说完自己的话后立即停止说话。    永远不要忘记将您的回答控制在 {word_limit} 个字以内！    不要添加其他内容。    """        )    )character_descriptions = [    generate_character_description(character_name) for character_name in character_names]character_system_messages = [    generate_character_system_message(character_name, character_description)    for character_name, character_description in zip(        character_names, character_descriptions    )]storyteller_specifier_prompt = [    player_descriptor_system_message,    HumanMessage(        content=f"""{game_description}        请用不超过 {word_limit} 个字回答以下问题: 请为讲故事者 {storyteller_name} 提供一个创意描述。         直接对 {storyteller_name} 说话。        不要添加其他内容。"""    ),]storyteller_description = ChatOpenAI(temperature=1.0)(    storyteller_specifier_prompt).contentstoryteller_system_message = SystemMessage(    content=(        f"""{game_description}您是讲故事者, {storyteller_name}. 您的描述如下: {storyteller_description}.其他玩家将提出行动并您将解释他们采取这些行动时会发生什么。请以第一人称从 {storyteller_name} 的角度说话。不要改变角色！不要以其他人的视角说话。记住您是讲故事者, {storyteller_name}。在您说完自己的话后立即停止说话。永远不要忘记将您的回答控制在 {word_limit} 个字以内！不要添加其他内容。"""    ))

In [6]:
# 打印故事讲述者描述print("故事讲述者描述:")print(storyteller_description)  # 打印故事讲述者的描述# 使用zip函数同时遍历角色名称和角色描述for character_name, character_description in zip(    character_names, character_descriptions):    print(f"{character_name} Description:")  # 打印角色名称和描述    print(character_description)  # 打印角色描述

Storyteller Description:
Dungeon Master, your power over this adventure is unparalleled. With your whimsical mind and impeccable storytelling, you guide us through the dangers of Hogwarts and beyond. We eagerly await your every twist, your every turn, in the hunt for Voldemort's cursed horcruxes.
Harry Potter Description:
"Welcome, Harry Potter. You are the young wizard with a lightning-shaped scar on your forehead. You possess brave and heroic qualities that will be essential on this perilous quest. Your destiny is not of your own choosing, but you must rise to the occasion and destroy the evil horcruxes. The wizarding world is counting on you."
Ron Weasley Description:
Ron Weasley, you are Harry's loyal friend and a talented wizard. You have a good heart but can be quick to anger. Keep your emotions in check as you journey to find the horcruxes. Your bravery will be tested, stay strong and focused.
Hermione Granger Description:
Hermione Granger, you are a brilliant and resourceful wi

## 使用LLM创建一个详细的任务描述LLM是指语言模型，可以用来生成任务描述。

In [7]:
# 定义一个包含系统消息和人类消息的列表quest_specifier_prompt = [    SystemMessage(content="You can make a task more specific."),  # 系统消息    HumanMessage(  # 人类消息        content=f"""{game_description}                You are the storyteller, {storyteller_name}.        Please make the quest more specific. Be creative and imaginative.        Please reply with the specified quest in {word_limit} words or less.         Speak directly to the characters: {*character_names,}.        Do not add anything else."""    ),]# 使用ChatOpenAI模型生成指定的任务specified_quest = ChatOpenAI(temperature=1.0)(quest_specifier_prompt).content# 打印原始任务和详细任务print(f"Original quest:\n{quest}\n")print(f"Detailed quest:\n{specified_quest}\n")

Original quest:
Find all of Lord Voldemort's seven horcruxes.

Detailed quest:
Harry Potter and his companions must journey to the Forbidden Forest, find the hidden entrance to Voldemort's secret lair, and retrieve the horcrux guarded by the deadly Acromantula, Aragog. Remember, time is of the essence as Voldemort's power grows stronger every day. Good luck.



## 主循环

In [8]:
# 创建一个空列表用于存储对话角色characters = []# 使用zip函数将character_names和character_system_messages进行遍历for character_name, character_system_message in zip(    character_names, character_system_messages):    # 将每个角色的信息添加到characters列表中    characters.append(        DialogueAgent(            name=character_name,  # 角色名称            system_message=character_system_message,  # 角色系统信息            model=ChatOpenAI(temperature=0.2),  # 使用ChatOpenAI模型创建对话角色        )    )# 创建一个讲故事的对话角色storyteller = DialogueAgent(    name=storyteller_name,  # 讲故事角色的名称    system_message=storyteller_system_message,  # 讲故事角色的系统信息    model=ChatOpenAI(temperature=0.2),  # 使用ChatOpenAI模型创建讲故事的对话角色)

In [9]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:    """    如果步骤是偶数，则选择讲故事的人    否则，以轮流的方式选择其他角色。    例如，有三个索引为1 2 3的角色    讲故事的人是索引0。    然后选择的索引将如下所示：    step: 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16    idx:  0  1  0  2  0  3  0  1  0  2  0  3  0  1  0  2  0    """    if step % 2 == 0:        idx = 0    else:        idx = (step // 2) % (len(agents) - 1) + 1    return idx

In [10]:
# 定义最大迭代次数max_iters = 20# 初始化迭代计数器n = 0# 创建对话模拟器，包括故事讲述者和其他角色，选择下一个发言者的函数为select_next_speakersimulator = DialogueSimulator(    agents=[storyteller] + characters, selection_function=select_next_speaker)# 重置模拟器simulator.reset()# 向模拟器注入故事讲述者的名字和指定的问题simulator.inject(storyteller_name, specified_quest)# 打印故事讲述者的名字和指定的问题print(f"({storyteller_name}): {specified_quest}")print("\n")# 迭代对话模拟器，直到达到最大迭代次数while n < max_iters:    # 模拟器进行一步对话，返回发言者的名字和发言内容    name, message = simulator.step()    # 打印发言者的名字和发言内容    print(f"({name}): {message}")    print("\n")    # 更新迭代计数器    n += 1

(Dungeon Master): Harry Potter and his companions must journey to the Forbidden Forest, find the hidden entrance to Voldemort's secret lair, and retrieve the horcrux guarded by the deadly Acromantula, Aragog. Remember, time is of the essence as Voldemort's power grows stronger every day. Good luck.


(Harry Potter): I suggest we sneak into the Forbidden Forest under the cover of darkness. Ron, Hermione, and I can use our wands to create a Disillusionment Charm to make us invisible. Filch, you can keep watch for any signs of danger. Let's move quickly and quietly.


(Dungeon Master): As you make your way through the Forbidden Forest, you hear the eerie sounds of nocturnal creatures. Suddenly, you come across a clearing where Aragog and his spider minions are waiting for you. Ron, Hermione, and Harry, you must use your wands to cast spells to fend off the spiders while Filch keeps watch. Be careful not to get bitten!


(Ron Weasley): I'll cast a spell to create a fiery blast to scare off